In [ ]:
!pwd

In [ ]:
def do_setup():
    # How to download competition data to temp folder(data) 
    # unzip it there, then symlink it like its a subdir
    # NOTE: make sure kaggle.json is in /root/.kaggle/

    #remove original symlink from this directory
    !rm ./data

    #create temp holder
    !mkdir /root/data

    #symlink it
    !ln -s /root/data ./data

    #download competition data to temp data folder
    !cd ./data;kaggle competitions download -c paddy-disease-classification

    #unzip it, -q is silent
    !cd ./data;unzip -q paddy-disease-classification.zip

    #setup dotfiles
    !wget "https://raw.githubusercontent.com/CNUClasses/dotfiles/master/setup.sh";chmod 766 setup.sh;source ./setup.sh  
    
import os
if(not os.path.exists('./data/train_images')):
   do_setup()  

In [ ]:
import pandas as pd
df =pd.read_csv('./data/train.csv')
df

In [ ]:
try: from path import Path
except ModuleNotFoundError:
    !pip install path
try: import timm
except ModuleNotFoundError:
    !pip install timm

In [ ]:
from fastai.vision.all import *
set_seed(42)

path =Path('./data/')

In [ ]:
path.ls()

In [ ]:
files=get_image_files(path/'train_images')
files

In [ ]:
img=PILImage.create(files[0])
print(img.size)
img.to_thumb(128)

In [ ]:
from fastcore.parallel import *
def f(o): return PILImage.create(o).size
sizes=parallel(f,files,n_workers=8)
pd.Series(sizes).value_counts()

In [ ]:
trnpath=path/'train_images'

In [ ]:
dls=ImageDataLoaders.from_folder(trnpath, valid_pct=0.2, seed=42,item_tfms=Resize(480, method='squish'),batch_tfms=aug_transforms(size=128, min_scale=0.75))
dls.show_batch(max_n=6)

In [ ]:
learn=vision_learner(dls,'resnet26d', metrics=error_rate,path='.')

In [ ]:
learn.lr_find(suggest_funcs=(valley,slide))

In [ ]:
learn.fine_tune(3,0.01)

### Try multihead

In [ ]:
#get the model thats part of learner
m=learn.model
# m

In [ ]:
#gets the head of the model
h=m[1]
h

In [ ]:
#subtract the last layer
ll=h[-1]
ll

In [ ]:
llp=list(ll.parameters())[0]

In [ ]:
#get rid of last linear layer in model
del(h[-1])
# m

In [ ]:
# learn.model

## Multihead

In [ ]:
dls=ImageDataLoaders.from_folder(trnpath, valid_pct=0.2, seed=42,item_tfms=Resize(480),batch_tfms=aug_transforms(size=224, min_scale=0.75))

In [ ]:
dblock=DataBlock(blocks=(ImageBlock, CategoryBlock),
                 get_items=get_image_files,
                 get_y=parent_label,
                 splitter=RandomSplitter(0.2, seed=42),
                 item_tfms=Resize(480,method='squish'),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75))

In [ ]:
path

In [ ]:
train_pth=path/'train_images'
train_pth

In [ ]:
dls=dblock.dataloaders(train_pth)
dls.show_batch()

In [ ]:
#gets all the image files in a folder
# len(get_image_files('./data'))

In [ ]:
import torch.nn as nn
class DiseaseAndTypeClassifier(nn.Module):
    def __init__(self,m):
        #lets strip the last layer out of the head
        del m[1][-1]
        super(DiseaseAndTypeClassifier,self).__init__()
        self.l1=nn.Linear(in_features=512, out_features=10, bias=False)  #rice type
        self.l2=nn.Linear(in_features=512, out_features=10, bias=False)  #disease
        self.m=m
        
    def forward(self,x):
        x=self.m(x)
        x1=self.l1(x)  #rice type
        x2=self.l2(x)  #disease
        return x1,x2
      

In [ ]:
# Learner??

In [ ]:
learn.loss_func
c_loss=learn.loss_func

In [ ]:
def dtc_loss(preds,targs):
    rice_preds, disease_preds=preds
    return c_loss(disease_preds,targs)

def dtc_error(preds,targs):
    rice_preds, disease_preds=preds
    return error_rate(disease_preds,targs)

In [ ]:
learn=vision_learner(dls,'resnet26d',  metrics=error_rate)
dtc=DiseaseAndTypeClassifier(learn.model)
learn.model=dtc
learn.loss_func=dtc_loss
learn.metrics=[dtc_error]

In [ ]:
preds=learn.get_preds(dl=learn.dls.valid)

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(1,0.01)

In [ ]:
#size of validation set?
len(learn.dls.valid.get_idxs())

In [ ]:
rice_preds, disease_preds=preds

In [ ]:
#copy and deepcopy is causing problems below
# from copy import copy,deepcopy
# arch='resnet26d'
# dls=ImageDataLoaders.from_folder(trnpath, valid_pct=0.2, seed=42,item_tfms=Resize(480),batch_tfms=aug_transforms(size=224, min_scale=0.75))
# learn3=vision_learner(dls,arch,metrics=error_rate)
# learn2=deepcopy(learn3)

### Old notebook

In [ ]:
llp.shape

In [ ]:
path

In [ ]:
ss=pd.read_csv(path/'sample_submission.csv')

In [ ]:
ss

In [ ]:
tst_files=get_image_files(path/'test_images').sorted()

tst_dl=dls.test_dl(tst_files)

In [ ]:
probs, bb, idxs=learn.get_preds(dl=tst_dl, with_decoded=True)
idxs

In [ ]:
dls.vocab

In [ ]:
d=dict(enumerate(dls.vocab))

In [ ]:
idxs_np=idxs.clone()

In [ ]:
idxs

In [ ]:
vals=pd.Series(data=idxs, name="idxs").map(d)
vals

In [ ]:
ss.label=vals

In [ ]:
ss

In [ ]:
ss.to_csv('subm.csv',index=False)

In [ ]:
!pwd

In [ ]:
from kaggle import api
comp = 'paddy-disease-classification'
api.competition_submit_cli('subm.csv', 'init', comp)